# Problema 3

Una empresa necesita asignar cuatro puestos de trabajo a cuatro trabajadores. El costo de desempeñar un puesto es una función de las habilidades de los trabajadores. En la tabla siguiente se resume el costo de las asignaciones. El trabajador 1 no puede tener el puesto 3, y el trabajador 3 no puede desempeñar el puesto 4. Determine la asignación óptima mediante programación lineal.

|              | Puesto 1 | Puesto 2 | Puesto 3 | Puesto 4 |
| ------------ | -------- | -------- | -------- | -------- |
| Trabajador 1 | \$50     | \$50     | –        | \$20     |
| Trabajador 2 | \$70     | \$40     | \$20     | \$30     |
| Trabajador 3 | \$90     | \$30     | \$50     | –        |
| Trabajador 4 | \$70     | \$20     | \$60     | \$70     |


### Cargar paquetes y definir datos

In [1]:
using JuMP
using HiGHS

# Crear el modelo
model = Model(HiGHS.Optimizer)

# Matriz de costos (trabajadores × puestos)
# Usamos `Inf` para los puestos prohibidos
costos = [
    50  50  Inf  20;
    70  40   20  30;
    90  30   50  Inf;
    70  20   60  70
]

m, n = size(costos)  # m = 4 trabajadores, n = 4 puestos


(4, 4)

### Variables y restricciones

In [2]:
# Variables binarias: x[i,j] = 1 si el trabajador i toma el puesto j
@variable(model, x[1:m, 1:n], Bin)

# Restricción: cada trabajador debe tener un puesto
for i in 1:m
    @constraint(model, sum(x[i,j] for j in 1:n) == 1)
end

# Restricción: cada puesto debe ser asignado a un solo trabajador
for j in 1:n
    @constraint(model, sum(x[i,j] for i in 1:m) == 1)
end


### Función objetivo, resolución y resultados

In [3]:
# Función objetivo: minimizar el costo total (evitando valores Inf)
@objective(model, Min, sum(costos[i,j] * x[i,j] for i in 1:m, j in 1:n if isfinite(costos[i,j])))

# Resolver el modelo
optimize!(model)

# Mostrar resultado de asignaciones
println("Asignaciones óptimas (trabajador → puesto):")
for i in 1:m
    for j in 1:n
        if isfinite(costos[i,j]) && value(x[i,j]) > 0.5
            println("Trabajador $i → Puesto $j (costo = \$", costos[i,j], ")")
        end
    end
end

println("Costo total mínimo: \$", objective_value(model))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 8 rows; 16 cols; 32 nonzeros; 16 integer variables (16 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 9e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 16 cols, 32 nonzeros  0s
8 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 0.1

Solving MIP model with:
   8 rows
   16 cols (16 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraint